In [6]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import numpy as np
from utils import *
import glob
import json
import datasets as ds

In [2]:
model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

/data/maryam.sana/anaconda3/envs/compressai/lib/python3.12/site-packages/transformers/models/llava/configuration_llava.py:104: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(
Loading checkpoint shards: 100%|█████████████████████████████████████████| 3/3 [00:02<00:00,  1.17it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def get_caption(frame):
    prompt = "<image>\nUSER:Give a detailed visual description of this image ?\nASSISTANT:"
    inputs = processor(text=prompt, images=frame, return_tensors="pt")
    generate_ids = model.generate(**inputs, max_length=200)
    caption_llava = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return caption_llava.split('ASSISTANT:')[1]  

In [5]:
img_files = sorted(glob.glob('data/Jockey/*.png'))
output_directory = 'data/custom_data/train/'
# Open a new JSON Lines file to write the metadata
with open('data/custom_data/train/metadata.jsonl', 'w') as file:
    for i, img_path in enumerate(img_files[0:10]):
        frame = Image.open(img_path)
        caption = get_caption(frame)
        new_img_path = f"{output_directory}{os.path.basename(img_path)}"
        # frame.save(new_img_path)

        # Prepare the dictionary to write into JSONL
        data = {
            "file_name": new_img_path.split('/')[-1],  # Extract filename from path
            "text": caption
        }

        # Write the JSON data to the file
        json.dump(data, file)
        file.write('\n')  # Newline for the next entry

print("Metadata file created successfully.")

Metadata file created successfully.


In [8]:
dataset = ds.load_dataset("imagefolder", data_dir='data/custom_data/', split="train")